In [92]:
import csv
import json
import pandas as pd
import re
from ast import literal_eval
import ast

In [93]:
df = pd.DataFrame({'BoolCol': [10, 20, 30, 30, 40],'attr': [22, 33, 22, 44, 66]},index=[1,2,3,4,5])

value_result = df[(df.BoolCol==20)&(df.attr==33)].index.to_list()
print(value_result[0])

2


In [94]:
raw_data = pd.read_csv('SteamAppDetail.csv', low_memory=False)

raw_data

,type,name,steam_appid,required_age,is_free,controller_support,dlc,fullgame,supported_languages,header_image,...,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Dota 2,570,0,True,NaN,"[1241930, 652720, 2120610]",NaN,"Bulgarian, Czech, Danish, Dutch, English<stron...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256692021, 'name': 'Dota 2 - Join the ...",{'total': 14308},NaN,"{'coming_soon': False, 'date': '9 Jul, 2013'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,game,Counter-Strike: Global Offensive,730,0,True,full,[1766730],NaN,"Czech, Danish, Dutch, English<strong>*</strong...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 81958, 'name': 'CS:GO Trailer Long', '...",{'total': 3500246},"{'total': 167, 'highlighted': [{'name': 'Someo...","{'coming_soon': False, 'date': 'Aug 21, 2012'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,PUBG: BATTLEGROUNDS,578080,0,True,NaN,NaN,NaN,"English, Korean, Simplified Chinese, French, G...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 1, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256896251, 'name': 'Battlegrounds_Dest...",{'total': 1625994},"{'total': 37, 'highlighted': [{'name': 'Last S...","{'coming_soon': False, 'date': 'Dec 21, 2017'}",{'url': 'https://support.playbattlegrounds.com...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': None}"
3,game,New World,1063730,0,False,NaN,NaN,NaN,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256888424, 'name': 'New World: Arenas'...",{'total': 200200},"{'total': 133, 'highlighted': [{'name': 'Repai...","{'coming_soon': False, 'date': 'Sep 28, 2021'}","{'url': 'https://www.amazon.com/contact-us', '...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'This Game may contai..."
4,game,Team Fortress 2,440,0,True,NaN,[629330],NaN,"English<strong>*</strong>, Danish, Dutch, Finn...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256698790, 'name': 'Jungle Inferno', '...",{'total': 15244},"{'total': 520, 'highlighted': [{'name': 'Head ...","{'coming_soon': False, 'date': 'Oct 10, 2007'}","{'url': 'http://steamcommunity.com/app/440', '...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes cartoon vio..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56722,game,Crawler,1506770,0,False,NaN,NaN,NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256826014, 'name': 'CrawTraiWL.mp4', '...",NaN,"{'total': 10, 'highlighted': [{'name': 'SECRET...","{'coming_soon': False, 'date': 'Apr 16, 2021'}","{'url': '', 'email': 'gamesmzy@gmail.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [5], 'notes': 'The game has low level ..."
56723,game,Coins Collector Simulator,186

In [95]:
# How many columns and Rows we have
print('We have ', raw_data.shape[0], "Rows in SteamAPI")
print('We have ', raw_data.shape[1], "Columns in SteamAPI\n")

null_counts = raw_data.isnull().sum() #How many null Columns and Rows we have
print('The empty rows in each Column:')
print(null_counts)

We have  56727 Rows in SteamAPI
We have  36 Columns in SteamAPI

The empty rows in each Column:
type                         100
name                         106
steam_appid                  100
required_age                 100
is_free                      100
controller_support         44741
dlc                        47266
fullgame                   56715
supported_languages          133
header_image                 100
website                    25450
pc_requirements              100
mac_requirements             100
linux_requirements           100
legal_notice               38943
drm_notice                 56425
ext_user_account_notice    55732
developers                   237
publishers                   100
demos                      51847
price_overview              8313
packages                    7721
package_groups               100
platforms                    100
metacritic                 52885
reviews                    47791
categories                  1079
genres       

In [96]:
'''
For the data from SteamAPI, we will keep the data which can be used. Those rows which misses names and steamappid will be cleaned.
We also find out that the rows with empty release_date were all removed after removing those with empty steamappid and names.
For the price_overview part, since we have another data source which is SteamSpy, we choose to keep them regardless of NAN.
'''

necessary_column = ['name', 'steam_appid', 'release_date']
optional_column = ['is_free', 'required_age']
for column in necessary_column:
    print('Rows to remove:', column, raw_data[raw_data[column].isnull()].shape[0])

Rows to remove: name 106
Rows to remove: steam_appid 100
Rows to remove: release_date 100


In [97]:
df = raw_data.drop(raw_data[raw_data['type'] != 'game'].index, inplace = False) #Remove all the rows with type not useable

column_keep = necessary_column + optional_column

df = df[column_keep] #only keep the columns we need

df = df.drop(df[df['name'].isnull() == True].index, inplace = False).reset_index(drop=True)

null_counts = df.isnull().sum() #How many null Columns and Rows we have
print(null_counts)
df

name            0
steam_appid     0
release_date    0
is_free         0
required_age    0
dtype: int64


,name,steam_appid,release_date,is_free,required_age
0,Dota 2,570,"{'coming_soon': False, 'date': '9 Jul, 2013'}",True,0
1,Counter-Strike: Global Offensive,730,"{'coming_soon': False, 'date': 'Aug 21, 2012'}",True,0
2,PUBG: BATTLEGROUNDS,578080,"{'coming_soon': False, 'date': 'Dec 21, 2017'}",True,0
3,New World,1063730,"{'coming_soon': False, 'date': 'Sep 28, 2021'}",False,0
4,Team Fortress 2,440,"{'coming_soon': False, 'date': 'Oct 10, 2007'}",True,0
...,...,...,...,...,...
56603,Crawler,1506770,"{'coming_soon': False, 'date': 'Apr 16, 2021'}",False,0
56604,Coins Collector Simulator,1868280,"{'coming_soon': False, 'date': 'Feb 4, 2022'}",False,0
56605,You Arrive in a Town,1466190,"{'coming_soon': False, 'date': 'Jan 26, 2021'}",False,0
56606,Hidden Treasures in the Forest of Dreams,1853990,"{'coming_soon': False, 'date': 'Jun 24, 2022'}",False,0


In [98]:
'''
After we remove the column names and steamappid with empty rows, we notice that the release_date column includes the 'Coming Soon', which we
should exclude in the final visualization
'''

index_coming_list = []
index_coming = 0
release_year = []
release_date = df['release_date']
for i in range(len(release_date)):
    release_date[i].strip()
    target = release_date[i][16:21]
    year = release_date[i][-6:-2]
    try:
        release_year.append(int(year))
    except:
        release_year.append(None)
    if target != 'False':
        index_coming_list.append(index_coming)
        release_year.pop()
    index_coming += 1

df = df.drop(df[df['name'].isnull() == True].index, inplace = False).reset_index(drop=True)

df = df.drop(index_coming_list, inplace = False).reset_index(drop=True)
df['release_date'] = release_year
df

,name,steam_appid,release_date,is_free,required_age
0,Dota 2,570,2013.0,True,0
1,Counter-Strike: Global Offensive,730,2012.0,True,0
2,PUBG: BATTLEGROUNDS,578080,2017.0,True,0
3,New World,1063730,2021.0,False,0
4,Team Fortress 2,440,2007.0,True,0
...,...,...,...,...,...
56484,Crawler,1506770,2021.0,False,0
56485,Coins Collector Simulator,1868280,2022.0,False,0
56486,You Arrive in a Town,1466190,2021.0,False,0
56487,Hidden Treasures in the Forest of Dreams,1853990,2022.0,False,0


In [99]:
'''
After cleaning most of the data in SteamAPI, the columns we want in SteamAPI have been filtered out.
However, in order to complete our visualization goals, we need more features including how many people own the game, the average rating on the game and the tags of the game.
In this case, we also mine data from the SteamSpy to provide a more comprehensive visualization
'''
raw_Spy = pd.read_csv('SteamSpyAppDetail.csv', low_memory = False)

raw_Spy.head(5)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,570,Dota 2,Valve,Valve,NaN,1508569,310425,0,"200,000,000 .. 500,000,000",37249,1678,870,955,0,0,0,"English, Bulgarian, Czech, Danish, Dutch, Finn...","Action, Free to Play, Strategy",681582,"{'Free to Play': 58585, 'MOBA': 19516, 'Multip..."
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,NaN,5881172,780421,0,"50,000,000 .. 100,000,000",29324,827,6220,265,0,0,0,"English, Czech, Danish, Dutch, Finnish, French...","Action, Free to Play",750651,"{'FPS': 88032, 'Shooter': 63261, 'Multiplayer'..."
2,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",NaN,1174490,904119,0,"50,000,000 .. 100,000,000",22185,846,6863,227,0,0,0,"English, Korean, Simplified Chinese, French, G...","Action, Adventure, Free to Play, Massively Mul...",301240,"{'Survival': 14119, 'Shooter': 11913, 'Multipl..."
3,1063730,New World,Amazon Games,Amazon Games,NaN,158670,74474,0,"50,000,000 .. 100,000,000",9459,831,3434,402,3999,3999,0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Massively Multiplayer, RPG",38753,"{'Massively Multiplayer': 583, 'Open World': 5..."
4,440,Team Fortress 2,Valve,Valve,NaN,838187,57152,0,"50,000,000 .. 100,000,000",9712,1450,359,350,0,0,0,"English, Danish, Dutch, Finnish, French, Germa...","Action, Free to Play",95148,"{'Free to Play': 61625, 'Hero Shooter': 60592,..."


In [100]:
# How many columns and Rows we have
print('We have ', raw_Spy.shape[0], "Rows in SteamSpy")
print('We have ', raw_Spy.shape[1], "Columns in SteamSpy\n")

null_counts = raw_Spy.isnull().sum() #How many null Columns and Rows we have
print('The empty rows in each Column:')
print(null_counts)

We have  56981 Rows in SteamSpy
We have  20 Columns in SteamSpy

The empty rows in each Column:
appid                  0
name                  11
developer            220
publisher            226
score_rank         56930
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                 29
initialprice          22
discount              22
languages             88
genre                251
ccu                    0
tags                   0
dtype: int64


In [101]:
'''
For the data from SteamSpy, we need to the similar thing we did in SteamAPI which is to keep the features we need.
'''

necessary_column = ['appid', 'positive', 'negative', 'owners', 'price', 'tags']
optional_column = ['name', 'publisher']
for column in necessary_column:
    print('Rows to remove:', column, raw_Spy[raw_Spy[column].isnull()].shape[0])

Rows to remove: appid 0
Rows to remove: positive 0
Rows to remove: negative 0
Rows to remove: owners 0
Rows to remove: price 29
Rows to remove: tags 0


In [102]:
df_spy = raw_Spy.drop(raw_Spy[raw_Spy['price'].isnull() == True].index, inplace = False).reset_index(drop=True)

df_spy = df_spy.drop(df_spy[df_spy['appid'] == 'appid'].index, inplace = False).reset_index(drop=True)

Spy_column_keep = necessary_column + optional_column

df_spy = df_spy[Spy_column_keep] #only keep the columns we need

df_spy

,appid,positive,negative,owners,price,tags,name,publisher
0,570,1508569,310425,"200,000,000 .. 500,000,000",0,"{'Free to Play': 58585, 'MOBA': 19516, 'Multip...",Dota 2,Valve
1,730,5881172,780421,"50,000,000 .. 100,000,000",0,"{'FPS': 88032, 'Shooter': 63261, 'Multiplayer'...",Counter-Strike: Global Offensive,Valve
2,578080,1174490,904119,"50,000,000 .. 100,000,000",0,"{'Survival': 14119, 'Shooter': 11913, 'Multipl...",PUBG: BATTLEGROUNDS,"KRAFTON, Inc."
3,1063730,158670,74474,"50,000,000 .. 100,000,000",3999,"{'Massively Multiplayer': 583, 'Open World': 5...",New World,Amazon Games
4,440,838187,57152,"50,000,000 .. 100,000,000",0,"{'Free to Play': 61625, 'Hero Shooter': 60592,...",Team Fortress 2,Valve
...,...,...,...,...,...,...,...,...
56941,1506770,5,7,"0 .. 20,000",399,"{'Adventure': 180, 'Action RPG': 153, 'Explora...",Crawler,MZY GAMES
56942,1868280,5,3,"0 .. 20,000",499,"{'Casual': 181, 'Racing': 174, 'Simulation': 1...",Coins Collector Simulator,"Atomic Fabrik, Cristian Manolachi"
56943,1466190,9,10,"0 .. 20,000",499,"{'Indie': 390, 'Action': 385, 'Casual': 382, '...",You Arrive in a Town,Bunny Box Games
56944,1853990,2,0,"0 .. 20,000",1999,"{'Relaxing': 252, 'Nature': 246, 'Walking Simu...",Hidden Treasures in the Forest of Dreams,Paleo Games LLC


In [103]:
df_spy = df_spy.drop(df_spy[df_spy['tags'] == '[]'].index, inplace = False).reset_index(drop=True)

tags = df_spy['tags']
tags

0        {'Free to Play': 58585, 'MOBA': 19516, 'Multip...
1        {'FPS': 88032, 'Shooter': 63261, 'Multiplayer'...
2        {'Survival': 14119, 'Shooter': 11913, 'Multipl...
3        {'Massively Multiplayer': 583, 'Open World': 5...
4        {'Free to Play': 61625, 'Hero Shooter': 60592,...
                               ...                        
56630    {'Adventure': 180, 'Action RPG': 153, 'Explora...
56631    {'Casual': 181, 'Racing': 174, 'Simulation': 1...
56632    {'Indie': 390, 'Action': 385, 'Casual': 382, '...
56633    {'Relaxing': 252, 'Nature': 246, 'Walking Simu...
56634    {'Early Access': 249, 'Fighting': 244, 'Arcade...
Name: tags, Length: 56635, dtype: object

In [104]:
'''
Deal with the tag object and split them into the list
'''

tag= []
for i in range(len(df_spy['tags'])):
    tag_object = df_spy['tags'][i][1:-1]
    try:
        tag_split = re.split('\:|\,',tag_object)
        temp = []
        for i in range(int((len(tag_split)-1)/2)):
            target = 2*i
            if i ==0 :
                temp.append(tag_split[target][1:-1])
            else:
                temp.append(tag_split[target][2:-1])
        tag.append(temp)
    except:
        tag.append([])
df_spy['tags'] = tag
df_spy

,appid,positive,negative,owners,price,tags,name,publisher
0,570,1508569,310425,"200,000,000 .. 500,000,000",0,"[Free to Play, MOBA, Multiplayer, Strategy, e-...",Dota 2,Valve
1,730,5881172,780421,"50,000,000 .. 100,000,000",0,"[FPS, Shooter, Multiplayer, Competitive, Actio...",Counter-Strike: Global Offensive,Valve
2,578080,1174490,904119,"50,000,000 .. 100,000,000",0,"[Survival, Shooter, Multiplayer, Battle Royale...",PUBG: BATTLEGROUNDS,"KRAFTON, Inc."
3,1063730,158670,74474,"50,000,000 .. 100,000,000",3999,"[Massively Multiplayer, Open World, MMORPG, RP...",New World,Amazon Games
4,440,838187,57152,"50,000,000 .. 100,000,000",0,"[Free to Play, Hero Shooter, Multiplayer, FPS,...",Team Fortress 2,Valve
...,...,...,...,...,...,...,...,...
56630,1506770,5,7,"0 .. 20,000",399,"[Adventure, Action RPG, Exploration, Dungeon C...",Crawler,MZY GAMES
56631,1868280,5,3,"0 .. 20,000",499,"[Casual, Racing, Simulation, Sports, Strategy,...",Coins Collector Simulator,"Atomic Fabrik, Cristian Manolachi"
56632,1466190,9,10,"0 .. 20,000",499,"[Indie, Action, Casual, RPG, Action RPG, Arcad...",You Arrive in a Town,Bunny Box Games
56633,1853990,2,0,"0 .. 20,000",1999,"[Relaxing, Nature, Walking Simulator, Procedur...",Hidden Treasures in the Forest of Dreams,Paleo Games LLC


In [105]:
print(df_spy.loc[df_spy['name'] == 'Counter-Strike'].tags)

29    [Action, FPS, Multiplayer, Shooter, Classic, T...
Name: tags, dtype: object


In [106]:
owners = []
for i in range(len(df_spy['owners'])):
    owner_object = df_spy['owners'][i]
    n1 = owner_object.split("..")[0].strip()
    n2 = owner_object.split("..")[1].strip()
    n1 = int(n1.replace(',' , ''))
    n2 = int(n2.replace(',' , ''))
    res = (n1+n2)/2
    owners.append(int(res))
df_spy['owners'] = owners
df_spy

,appid,positive,negative,owners,price,tags,name,publisher
0,570,1508569,310425,350000000,0,"[Free to Play, MOBA, Multiplayer, Strategy, e-...",Dota 2,Valve
1,730,5881172,780421,75000000,0,"[FPS, Shooter, Multiplayer, Competitive, Actio...",Counter-Strike: Global Offensive,Valve
2,578080,1174490,904119,75000000,0,"[Survival, Shooter, Multiplayer, Battle Royale...",PUBG: BATTLEGROUNDS,"KRAFTON, Inc."
3,1063730,158670,74474,75000000,3999,"[Massively Multiplayer, Open World, MMORPG, RP...",New World,Amazon Games
4,440,838187,57152,75000000,0,"[Free to Play, Hero Shooter, Multiplayer, FPS,...",Team Fortress 2,Valve
...,...,...,...,...,...,...,...,...
56630,1506770,5,7,10000,399,"[Adventure, Action RPG, Exploration, Dungeon C...",Crawler,MZY GAMES
56631,1868280,5,3,10000,499,"[Casual, Racing, Simulation, Sports, Strategy,...",Coins Collector Simulator,"Atomic Fabrik, Cristian Manolachi"
56632,1466190,9,10,10000,499,"[Indie, Action, Casual, RPG, Action RPG, Arcad...",You Arrive in a Town,Bunny Box Games
56633,1853990,2,0,10000,1999,"[Relaxing, Nature, Walking Simulator, Procedur...",Hidden Treasures in the Forest of Dreams,Paleo Games LLC


In [107]:
rate = []
for i in range(len(df_spy['positive'])):
    positive_rate = int(df_spy['positive'][i])
    negative_rate = int(df_spy['negative'][i])
    if positive_rate+negative_rate == 0:
        res = None
    else:
        res = positive_rate/(positive_rate+negative_rate)
    rate.append(res)
df_spy['rate'] = rate
df_spy

,appid,positive,negative,owners,price,tags,name,publisher,rate
0,570,1508569,310425,350000000,0,"[Free to Play, MOBA, Multiplayer, Strategy, e-...",Dota 2,Valve,0.829342
1,730,5881172,780421,75000000,0,"[FPS, Shooter, Multiplayer, Competitive, Actio...",Counter-Strike: Global Offensive,Valve,0.882848
2,578080,1174490,904119,75000000,0,"[Survival, Shooter, Multiplayer, Battle Royale...",PUBG: BATTLEGROUNDS,"KRAFTON, Inc.",0.565037
3,1063730,158670,74474,75000000,3999,"[Massively Multiplayer, Open World, MMORPG, RP...",New World,Amazon Games,0.680567
4,440,838187,57152,75000000,0,"[Free to Play, Hero Shooter, Multiplayer, FPS,...",Team Fortress 2,Valve,0.936167
...,...,...,...,...,...,...,...,...,...
56630,1506770,5,7,10000,399,"[Adventure, Action RPG, Exploration, Dungeon C...",Crawler,MZY GAMES,0.416667
56631,1868280,5,3,10000,499,"[Casual, Racing, Simulation, Sports, Strategy,...",Coins Collector Simulator,"Atomic Fabrik, Cristian Manolachi",0.625000
56632,1466190,9,10,10000,499,"[Indie, Action, Casual, RPG, Action RPG, Arcad...",You Arrive in a Town,Bunny Box Games,0.473684
56633,1853990,2,0,10000,1999,"[Relaxing, Nature, Walking Simulator, Procedur...",Hidden Treasures in the Forest of Dreams,Paleo Games LLC,1.000000


In [108]:
miss_count = 0
miss_index = []
hit_index = []
rate_list = []
price_list = []
owners_list = []
tags_list = []
for i in range(len(df['steam_appid'])):
    find_flag = 1
    target = df['steam_appid'][i]
    res = df_spy[df_spy.appid == target].index.to_list()
    if res == []:
        find_flag = 0
    else:
        hit_index.append(res[0])
    if i % 100 == 0:
        print('We now on the ' + str(i) +' steam_appid')
    if find_flag == 0:
        miss_count += 1
        hit_index.append(0)
        miss_index.append(i)
    else:
        continue
print(miss_count)

We now on the 0 steam_appid
We now on the 100 steam_appid
We now on the 200 steam_appid
We now on the 300 steam_appid
We now on the 400 steam_appid
We now on the 500 steam_appid
We now on the 600 steam_appid
We now on the 700 steam_appid
We now on the 800 steam_appid
We now on the 900 steam_appid
We now on the 1000 steam_appid
We now on the 1100 steam_appid
We now on the 1200 steam_appid
We now on the 1300 steam_appid
We now on the 1400 steam_appid
We now on the 1500 steam_appid
We now on the 1600 steam_appid
We now on the 1700 steam_appid
We now on the 1800 steam_appid
We now on the 1900 steam_appid
We now on the 2000 steam_appid
We now on the 2100 steam_appid
We now on the 2200 steam_appid
We now on the 2300 steam_appid
We now on the 2400 steam_appid
We now on the 2500 steam_appid
We now on the 2600 steam_appid
We now on the 2700 steam_appid
We now on the 2800 steam_appid
We now on the 2900 steam_appid
We now on the 3000 steam_appid
We now on the 3100 steam_appid
We now on the 3200 s

In [109]:
for i in hit_index:
    rate_list.append(df_spy['rate'][i])
    price_list.append(int(df_spy['price'][i]))
    owners_list.append(df_spy['owners'][i])
    tags_list.append(df_spy['tags'][i])
df['rate'] = rate_list
df['price'] = price_list
df['owners'] = owners_list
df['tags'] = tags_list

df = df.drop(labels = miss_index, axis = 0, inplace=False).reset_index(drop=True)
df

,name,steam_appid,release_date,is_free,required_age,rate,price,owners,tags
0,Dota 2,570,2013.0,True,0,0.829342,0,350000000,"[Free to Play, MOBA, Multiplayer, Strategy, e-..."
1,Counter-Strike: Global Offensive,730,2012.0,True,0,0.882848,0,75000000,"[FPS, Shooter, Multiplayer, Competitive, Actio..."
2,PUBG: BATTLEGROUNDS,578080,2017.0,True,0,0.565037,0,75000000,"[Survival, Shooter, Multiplayer, Battle Royale..."
3,New World,1063730,2021.0,False,0,0.680567,3999,75000000,"[Massively Multiplayer, Open World, MMORPG, RP..."
4,Team Fortress 2,440,2007.0,True,0,0.936167,0,75000000,"[Free to Play, Hero Shooter, Multiplayer, FPS,..."
...,...,...,...,...,...,...,...,...,...
56341,Crawler,1506770,2021.0,False,0,0.416667,399,10000,"[Adventure, Action RPG, Exploration, Dungeon C..."
56342,Coins Collector Simulator,1868280,2022.0,False,0,0.625000,499,10000,"[Casual, Racing, Simulation, Sports, Strategy,..."
56343,You Arrive in a Town,1466190,2021.0,False,0,0.473684,499,10000,"[Indie, Action, Casual, RPG, Action RPG, Arcad..."
56344,Hidden Treasures in the Forest of Dreams,1853990,2022.0,False,0,1.000000,1999,10000,"[Relaxing, Nature, Walking Simulator, Procedur..."


In [110]:
'''
We need to remove all the Early Access tag in the tags because Early Access can not be counted as a game genre
'''
early_index = []
for i in range(len(df['tags'])):
    if "Early Access" in df['tags'][i]:
        early_index.append(i)
df = df.drop(labels = early_index, axis = 0, inplace=False).reset_index(drop=True)


In [111]:
df = df.drop(df[df['release_date'].isnull() ==True].index, axis = 0, inplace=False).reset_index(drop=True)

df['release_date'] = df['release_date'].astype(int)

In [112]:
df

,name,steam_appid,release_date,is_free,required_age,rate,price,owners,tags
0,Dota 2,570,2013,True,0,0.829342,0,350000000,"[Free to Play, MOBA, Multiplayer, Strategy, e-..."
1,Counter-Strike: Global Offensive,730,2012,True,0,0.882848,0,75000000,"[FPS, Shooter, Multiplayer, Competitive, Actio..."
2,New World,1063730,2021,False,0,0.680567,3999,75000000,"[Massively Multiplayer, Open World, MMORPG, RP..."
3,Team Fortress 2,440,2007,True,0,0.936167,0,75000000,"[Free to Play, Hero Shooter, Multiplayer, FPS,..."
4,Apex Legends™,1172470,2020,True,0,0.858303,0,75000000,"[Free to Play, Multiplayer, Battle Royale, Sho..."
...,...,...,...,...,...,...,...,...,...
49541,VEHICLES FURY,749290,2017,False,0,0.166667,1499,10000,"[Action, Indie, Racing]"
49542,Crawler,1506770,2021,False,0,0.416667,399,10000,"[Adventure, Action RPG, Exploration, Dungeon C..."
49543,Coins Collector Simulator,1868280,2022,False,0,0.625000,499,10000,"[Casual, Racing, Simulation, Sports, Strategy,..."
49544,You Arrive in a Town,1466190,2021,False,0,0.473684,499,10000,"[Indie, Action, Casual, RPG, Action RPG, Arcad..."


In [113]:
print(type(df['release_date'][0]))

<class 'numpy.int32'>


In [116]:
df.to_csv('data.csv')

In [115]:
print(df['tags'][0])

['Free to Play', 'MOBA', 'Multiplayer', 'Strategy', 'e-sports', 'Team-Based', 'Competitive', 'Action', 'Online Co-Op', 'PvP', 'Difficult', 'Co-op', 'RTS', 'Tower Defense', 'Fantasy', 'RPG', 'Character Customization', 'Replay Value', 'Action RPG']
